In [1]:
import csv

In [2]:
count = 0
with open("Questions.csv",encoding="latin1") as csvfile:
    csv_file = csv.reader(csvfile)
    next(csv_file,None)
    for row in csv_file:
        count +=1
print(count," , ",len(row)) 

1264216  ,  7


In [3]:
#!pip install elasticsearch

In [4]:
## Creating Sentence Vector

In [5]:
#!pip install tensorflow

In [6]:
import tensorflow as tf
import tensorflow_hub as hub

In [7]:
sent_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [8]:
sent_embeddings = tf.make_ndarray(tf.make_tensor_proto(sent_embed(["In the end Trust is very funny"]))).tolist()[0]
print(type(sent_embeddings))
print(len(sent_embeddings))
print(sent_embeddings)

<class 'list'>
512
[0.005338217131793499, 0.018591906875371933, -0.04280121251940727, 0.02388177067041397, 0.028391873463988304, 0.01152508519589901, 0.01682991534471512, 0.0036806624848395586, 0.02447395958006382, 0.03715682774782181, -0.0039298334158957005, -0.027621513232588768, 0.006198609713464975, 0.04287201911211014, 0.04297664761543274, -0.10033304244279861, 0.04333379492163658, -0.0017588174669072032, 0.046127140522003174, 0.04848652705550194, -0.01832684874534607, -0.04373626783490181, 0.07402496039867401, 0.0018099364824593067, 0.018523553386330605, 0.02012397162616253, -0.016803869977593422, 0.029332226142287254, -0.011031173169612885, 0.04836065322160721, 0.005182396154850721, 0.04673437401652336, -0.02262251451611519, -0.0439649298787117, -0.03720908239483833, -0.022510599344968796, -0.09106342494487762, 0.014294910244643688, 0.012142586521804333, -0.041628241539001465, 0.07088243216276169, -0.08002688735723495, -0.05365167558193207, 0.08674485981464386, -0.03194680809974

In [9]:
import json
import time
import sys
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import csv
# connect to ES on localhost on port 9200
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
if es.ping():
	print('Connected to ES!')
else:
	print('Could not connect!')
	sys.exit()

print("*********************************************************************************");


# index in ES is similar to creating a DB
# Read each question and index into an index called questions
# Indexing only titles as a POC
# Define the index


# Mapping: Structure of the index
# Property/Field: name and type  these are basically columns of the index
b = {"mappings": {
  	"properties": {
    		"title": {
      			"type": "text"
    		},
    		"title_vector": {
      			"type": "dense_vector",
      			"dims": 512
		}
	}
     }
   }


ret = es.indices.create(index='questions-index_1', ignore=400, body=b) #400 caused by IndexAlreadyExistsException, 
print(json.dumps(ret,indent=4))

# TRY this in browser: http://localhost:9200/questions-index

print("*********************************************************************************");

#load USE4 model

#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")




# CONSTANTS
NUM_QUESTIONS_INDEXED = 100000

# Col-Names: Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
cnt=0

with open('Questions.csv', encoding="latin1") as csvfile:
	readCSV = csv.reader(csvfile, delimiter=',' )
	next(readCSV, None)  # skip the headers 
	for row in readCSV:
		#print(row[0], row[5])
		doc_id = row[0];
		title = row[5];
		vec = tf.make_ndarray(tf.make_tensor_proto(sent_embed([title]))).tolist()[0]		
		
		b = {"title":title,
			"title_vector":vec,
			}	
		#print(json.dumps(tmp,indent=4))		
		
		res = es.index(index="questions-index", id=doc_id, body=b)
		#print(res)
		

		# keep count of # rows processed
		cnt += 1
		if cnt%100==0:
			print(cnt)
		
		if cnt == NUM_QUESTIONS_INDEXED:
			break;

	print("Completed indexing....")

	print("*********************************************************************************");
	





Connected to ES!
*********************************************************************************
{
    "error": {
        "root_cause": [
            {
                "type": "resource_already_exists_exception",
                "reason": "index [questions-index_1/SPdRB6e7SeCbwHI3vbaNhA] already exists",
                "index_uuid": "SPdRB6e7SeCbwHI3vbaNhA",
                "index": "questions-index_1"
            }
        ],
        "type": "resource_already_exists_exception",
        "reason": "index [questions-index_1/SPdRB6e7SeCbwHI3vbaNhA] already exists",
        "index_uuid": "SPdRB6e7SeCbwHI3vbaNhA",
        "index": "questions-index_1"
    },
    "status": 400
}
*********************************************************************************
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
490

In [ ]:
def connect2ES():
    # connect to ES on localhost on port 9200
    es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if es.ping():
            print('Connected to ES!')
    else:
            print('Could not connect!')
            sys.exit()

    print("*********************************************************************************");
    return es

# def keywordSearch(es, q):
#     #Search by Keywords
#     b={
#             'query':{
#                 'match':{
#                     "title":q
#                 }
#             }
#         }

#     res= es.search(index='questions-index_1',body=b)
#     print(res)
#     print("Keyword Search:\n")
#     print("hello_out")
#     for hit in res['hits']['hits']:
#         print(hello_in)
#         print(str(hit['_score']) + "\t" + hit['_source']['title'] )

#     print("*********************************************************************************");

#     return


# Search by Vec Similarity
def sentenceSimilaritybyNN(embed, es, sent):
    query_vector = tf.make_ndarray(tf.make_tensor_proto(sent_embed([sent]))).tolist()[0]
    b = {"query" : {
                "script_score" : {
                    "query" : {
                        "match_all": {}
                    },
                    "script" : {
                        "source": "cosineSimilarity(params.query_vector, 'title_vector') + 1.0",
                        "params": {"query_vector": query_vector}
                    }
                }
             }
        }


    #print(json.dumps(b,indent=4))
    res= es.search(index='questions-index',body=b)

    print("Semantic Similarity Search:\n")
    for hit in res['hits']['hits']:
        print(str(hit['_score']) + "\t" + hit['_source']['title'] )

    print("*********************************************************************************");



if __name__=="__main__":

    es = connect2ES();
    sent_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

    while(1):
        query=input("Enter a Query:");

        start = time.time()
        if query=="END":
            break;

        print("Query: " +query)
        #keywordSearch(es, query)
        sentenceSimilaritybyNN(sent_embed, es, query)

        end = time.time()
        print(end - start)

Connected to ES!
*********************************************************************************
Enter a Query:read a file in python
Query: read a file in python
Semantic Similarity Search:

1.8942376	Read and overwrite a file in Python
1.8832392	Reading from files in python
1.8228974	Read file object as string in python
1.82159	Python: Read a file (from an external server)
1.8138329	reading .bash_history file through python script
1.8127863	Reading bytestreams in Python
1.7825418	Python, find a file in the same directory
1.7734303	Take user input and put it into a file in Python?
1.766594	access a file in python that is created from SunGridEngine
1.7610618	Reading file using python and and see if a particular string is there inthe file
*********************************************************************************
2.429131031036377
Enter a Query:how to install python
Query: how to install python
Semantic Similarity Search:

1.8684404	How to install pyobjc on SnowLeopard's non-defa